# Power Plant Modelling

In [1]:
# Import useful libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import sklearn
import datetime
import yfinance as yf
import seaborn as sns
import random
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.signal import StateSpace
from typing import Optional, List
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

import pandas as pd
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


In [2]:
# Useful functions

def information(input):
    print("The array is : \n", input)
    print(" ")
    print("type:", type(input))
    print("sys.getsizeof (Python object size) KB:", sys.getsizeof(input) / (1024))
    print("dtype:", input.dtype)
    print("shape:", input.shape)
    print(" ")
        

## State Space models

Using standard class definition as suggested


def __init__ ( self, **params):

"""

Initialize the digital twin with optional model paramters


Paramters (optional):

param1(type) : Description of the parameter

param1(type) : Description of the parameter

"""

self.params = params    

def __call__(self, inputs: dict) -> dict :

"""

Execute the digital twin simulation based on input variables

Expected inputs:

    - input1 (type) : Description

    - input2 (type) : Description

Returns : 

    dict with:

        - output1 (type) : Description

        - output2 (type) : Description


"""

Extract inputs with deafult values

input1 = inputs.get("input1", 0)

input2 = inputs.get("input2", 0)

output1 = f(input1)

output2 = g(input2)

return {

    "output1": output1,

    "output2": output2 
    
}


We describe the system using a set of first-order differential equations:

$$
\begin{aligned}
\dot{x}(t) &= A x(t) + B u(t) \\
y(t) &= C x(t) + D u(t)
\end{aligned}
$$

or 

$$
\begin{aligned}
x_{k+1} &= A x_k + B u_k \\
y_k &= C x_k + D u_k
\end{aligned}
$$


Where:

- $ x \in \mathbb{R}^n $ is the **state vector**
- $ u \in \mathbb{R}^m $ is the **input vector**
- $ y \in \mathbb{R}^p $ is the **output vector**
- $ A \in \mathbb{R}^{n \times n} $ is the **system matrix**
- $ B \in \mathbb{R}^{n \times m} $ is the **input matrix**
- $ C \in \mathbb{R}^{p \times n} $ is the **output matrix**
- $ D \in \mathbb{R}^{p \times m} $ is the **feedthrough matrix**

In our model 

- $ x \in \mathbb{R}^2 $ is the **state vector**
- $ u \in \mathbb{R}^6 $ is the **input vector**
- $ y \in \mathbb{R}^1 $ is the **output vector**
- $ A \in \mathbb{R}^{2 \times 2} $ is the **system matrix**
- $ B \in \mathbb{R}^{2 \times 6} $ is the **input matrix**
- $ C \in \mathbb{R}^{1 \times 2} $ is the **output matrix**
- $ D \in \mathbb{R}^{1 \times 6} $ is the **feedthrough matrix**

In particular : 
$$
u_k = \begin{bmatrix}
u^1_k \\
u^2_k \\
u^3_k \\
u^4_k \\
u^5_k \\
u^6_k
\end{bmatrix}
$$


- $ u^i_k \in \mathbb{R} $ = input power group i at step $k$ $[W]$


and $$ u^{i}_k = NCV_{\text{BFG}} \; {\dot{v}_k}_{\text{BFG}}^{g_i} + NCV_{\text{BOFG}} \; {\dot{v}_k}_{\text{BOFG}}^{g_i} + NCV_{\text{NG}} \; {\dot{v}_k}_{\text{NG}}^{g_i} + NCV_{\text{COG}} \; {\dot{v}_k}_{\text{COG}}^{g_i} \; \; \; \; [W]$$

and $$ y_k = \text{output power PP} \; \; \; \; [W]$$ 


In [ ]:
# Define PP state space
# Define A,B,C,D
A_PP = np.array([
    [0.9275, -0.3135],
    [-0.0856, 0.2482]
])

B_PP = 1.0e-03 * np.array([
    [0.0374, 0.0544, 0.3203, 0.1426, -0.0568, 0.1194],
    [0.0730, 0.0962, 0.7956, 0.2623, -0.1350, 0.2562]
])

C_PP = 1.0e-03 * np.array([
    [1.2313, -0.0577]
])

D_PP = np.zeros((1, 6))

# Initial state
x_PP = np.array([
    0.2180,
    0.1180  
])

print("A is :")
information(A_PP)
print("\n")

print("B is :")
information(B_PP)
print("\n")

print("C is :")
information(C_PP)
print("\n")


A is :
The array is : 
 [[ 0.9275 -0.3135]
 [-0.0856  0.2482]]
 
type: <class 'numpy.ndarray'>
sys.getsizeof (Python object size) KB: 0.15625
dtype: float64
shape: (2, 2)
 


B is :
The array is : 
 [[ 3.740e-05  5.440e-05  3.203e-04  1.426e-04 -5.680e-05  1.194e-04]
 [ 7.300e-05  9.620e-05  7.956e-04  2.623e-04 -1.350e-04  2.562e-04]]
 
type: <class 'numpy.ndarray'>
sys.getsizeof (Python object size) KB: 0.21875
dtype: float64
shape: (2, 6)
 


C is :
The array is : 
 [[ 1.2313e-03 -5.7700e-05]]
 
type: <class 'numpy.ndarray'>
sys.getsizeof (Python object size) KB: 0.140625
dtype: float64
shape: (1, 2)
 




In [ ]:
class PowerPlant :
    """
    Class describing State space Power plant ouput power
    
    Parameters :
    
    State dimension : n = 2, 
    Input dimension : m = 6, 
    Output dimensions : p = 1
    
    A : n x n (ndarray, dtype = float)
    B : n x m (ndarray, dtype = float)
    C : p x n (ndarray, dtype = float)
    D : p x m (ndarray, dtype = float)
    x0: n x 1 (ndarray, dtype = float)
    
    Input : 
    u : m dimensional vector containing input power [MW]
    
    Output :
    y : p dimensional vector containing output Powerplant power [MW]

    """
    
    def __init__(self, A : np.ndarray = A_PP  , B : np.ndarray = B_PP, C : np.ndarray = C_PP, D : np.ndarray = D_PP, x0 : np.ndarray = x_PP, input_names : Optional[List[str]] = None, output_names : Optional[List[str]] = None) :
        """
        
        Parameters : 
        
        A : n x n (ndarray, dtype = float)
        B : n x m (ndarray, dtype = float)
        C : p x n (ndarray, dtype = float)
        D : p x m (ndarray, dtype = float)
        x0: n x 1 (ndarray, dtype = float)
        input_names : List of strings ( m dimensional ) ; type([List[str]])
        output_names : List of strings ( p dimensional ) ; type([List[str]])
        
        """
        
        # Declare state space
        self.A = np.array(A, dtype = float)
        self.B = np.array(B, dtype = float)
        self.C = np.array(C, dtype = float)
        self.D = np.array(D, dtype = float)
        self.x = np.array(x0, dtype = float)
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # Check
        assert self.A.shape == (n,n), f" A must be a square matrix {n} x {n}"
        assert self.B.shape == (n,m), f" B must be a matrix {n} x {m}"
        assert self.C.shape == (p,n), f" C must be a matrix {p} x {n}"
        assert self.D.shape == (p,m), f" D must be a matrix {p} x {m}"
        
        self.input_names = input_names if input_names else [f"u{i}" for i in range(m)]
        self.output_names = output_names if output_names else [f"y{i}" for i in range(p)]
        
    
    def __call__(self, input_dict : dict) -> dict:
        """
        
        Compute output y_{k} = C * x_{k} + D * u_{k} given current state and input.
        Compute  x_{k+1} = A * x_{k} + B * u_{k} given current state and input.
        
        input_dict: {input_name: value} --> {input_name_1 : value_1, input_name_2 : value_2, ..., }
        return: {output_name: value} --> {output_name_1 : value_1, output_name_2 : value_2, ...., }
        
        """
        
        # declare dimensions
        n, m = self.B.shape
        p, _ = self.C.shape
        
        # create u vector of m dimension with values in input names
        u = np.array ( [input_dict[name] for name in self.input_names ], dtype = float )
        
        assert u.shape == (m,), f"u must be {m} dimensional"
        
        # update x_{k+1} = A * x_{k} + B * u_{k}
        self.x = self.A @ self.x + self.B @ u
        
        assert self.x.shape == (n,), f"x must be {n} dimensional"
        
        y = self.C @ self.x + self.D @ u
        
        assert self.y.shape == (p,), f"y must be {p} dimensional"
        
        return {name: y[i] for i, name in enumerate(self.output_names) }